In [27]:
import numpy as np
import string
import json
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, Bidirectional, GlobalMaxPool1D
from keras.layers.core import SpatialDropout1D
from sklearn.model_selection import StratifiedKFold
from keras.datasets import imdb
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import model_from_json

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

from gensim import corpora
from imblearn.over_sampling import SMOTE

# def parsePhrases(stopWords, engStemmer, phrases):
#     print "parse the phrases with stopwords and stemmer"
#     processedPhrases = []
#     for phrase in phrases:
#         tokens = word_tokenize(phrase)
#         parsedWords = []
#         for t in tokens:
#             if t not in stopWords:
#                 parsedWords.append(engStemmer.stem(t))
#         processedPhrases.append(parsedWords)
#     return processedPhrases
postProcessedTrainPhrases = []
postProcessedTestPhrases = []

def preprocessData():
    print("Loading and preprocessing data...")
    # load training and testing data
    with open('labeled_document2.json') as json_data:
        allTrainData = json.load(json_data)


    trainPhrases, testPhrases, trainLabel,testLabel = train_test_split(allTrainData['Comment'], allTrainData['CommentLabel'], test_size=0.2, random_state=42)
    
#     print(testPhrases[0:100])
    punctuation = list(string.punctuation)
    stopWords = stopwords.words('english') + punctuation 

    engStemmer = SnowballStemmer('english')
    for phrase in trainPhrases:
        if not isinstance(phrase, str):
            continue
        tokens = word_tokenize(phrase)
        parsedWords = []
        for t in tokens:
            if t not in stopWords:
                parsedWords.append(engStemmer.stem(t))
        postProcessedTrainPhrases.append(parsedWords)

    for phrase in testPhrases:
        if not isinstance(phrase, str):
            continue
        tokens = word_tokenize(phrase)
        parsedWords = []
        for t in tokens:
            if t not in stopWords:
                parsedWords.append(engStemmer.stem(t))
        postProcessedTestPhrases.append(parsedWords)
    return (trainLabel,testLabel)


def convertPhrasesToIDs(phrases):
    print ("converting the phrases to id to be processed")
    wordIDs = []
    wordIDLens = []
    for phrase in phrases:
        ids = []
        for word in phrase:
            ids.append(toIDMap.token2id[word])
        wordIDs.append(ids)
        wordIDLens.append(len(ids))
    return ( wordIDs, wordIDLens )

def findSequenceLen(wordListLen):
    print( "calculate the norm sequence length")
    wordLenMean = np.mean(wordListLen)
    wordLenStd = np.std(wordListLen)
    return np.round(wordLenMean + 3 * wordLenStd).astype(int)



In [28]:
(trainSenti, testSenti) = preprocessData()
# process training data and testing data

print(len(postProcessedTrainPhrases), len(trainSenti))
toIDMap = corpora.Dictionary(np.concatenate((postProcessedTrainPhrases, postProcessedTestPhrases), axis=0))
allPhraseSize = len(toIDMap.keys())

(trainWordIDs, trainWordIDLens) = convertPhrasesToIDs(postProcessedTrainPhrases)
(testWordIDs, testWordIDLens) = convertPhrasesToIDs(postProcessedTestPhrases)

sequenceLen = findSequenceLen(trainWordIDLens + testWordIDLens)

print( "pad sequence")
trainingData = sequence.pad_sequences(np.array(trainWordIDs), maxlen=sequenceLen)
testingData = sequence.pad_sequences(np.array(testWordIDs), maxlen=sequenceLen)
print(trainingData.shape)

print ("categorize the labels")
#print len(np.unique(trainSenti))
trainingDataLabel = np_utils.to_categorical(trainSenti, len(np.unique(trainSenti)))

print(trainingDataLabel.shape)

# model = Sequential()
# model.add(Embedding(allPhraseSize, 128))
# model.add(SpatialDropout1D(0.1))
# model.add(Bidirectional(LSTM(128)))
# #model.add(Bidirectional(LSTM(128)))
# #model.add(Flatten())
# model.add(Dense(len(np.unique(trainSenti))))
# model.add(Activation('softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model = Sequential()
# model.add(Embedding(allPhraseSize, 128, dropout=0.2))
# model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2))
# model.add(Dense(num_labels))
# model.add(Activation('softmax'))

model = Sequential()
model.add(Embedding(allPhraseSize, 128))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.1))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(len(np.unique(trainSenti))))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


model.fit(trainingData,trainingDataLabel , epochs=10, batch_size=256, verbose=1)
# evaluate the model
testingDataLabel = np_utils.to_categorical(testSenti, len(np.unique(testSenti)))
scores = model.evaluate(testingData, testingDataLabel, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))

Loading and preprocessing data...
3394 3394
converting the phrases to id to be processed
converting the phrases to id to be processed
calculate the norm sequence length
pad sequence
(3394, 46)
categorize the labels
(3394, 2)
Epoch 1/10
3394/3394 [==============================] - 19s 5ms/step - loss: 0.5416 - acc: 0.8250
Epoch 2/10
3394/3394 [==============================] - 14s 4ms/step - loss: 0.4514 - acc: 0.8385
Epoch 3/10
3394/3394 [==============================] - 14s 4ms/step - loss: 0.4322 - acc: 0.8385
Epoch 4/10
3394/3394 [==============================] - 13s 4ms/step - loss: 0.3946 - acc: 0.8385
Epoch 5/10
3394/3394 [==============================] - 18s 5ms/step - loss: 0.3056 - acc: 0.8406
Epoch 6/10
3394/3394 [==============================] - 17s 5ms/step - loss: 0.2157 - acc: 0.9093
Epoch 7/10
3394/3394 [==============================] - 16s 5ms/step - loss: 0.1433 - acc: 0.9467
Epoch 8/10
3394/3394 [==============================] - 15s 4ms/step - loss: 0.1045 - acc

In [23]:
print(trainingData.shape, trainingDataLabel.shape)

(3394, 46) (3394, 2)


In [26]:
print(scores)

[0.9333925036575264, 0.805653710457967]


In [32]:
predictedRes = model.predict_proba(testingData)

In [29]:
model_json = model.to_json()
with open("LSTM.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("LSTM.h5")
print("Saved model to disk")


Saved model to disk


In [37]:
json_file = open('LSTM.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("LSTM.h5")
print("Loaded model from disk")

Loaded model from disk


In [38]:
predict_res = loaded_model.predict_proba(testingData)

In [39]:
print(predict_res)

[[0.33525658 0.6647435 ]
 [0.06602536 0.9339746 ]
 [0.00454879 0.9954513 ]
 ...
 [0.8005882  0.19941178]
 [0.9947694  0.00523055]
 [0.693177   0.30682302]]


In [ ]:
acc: 80.57%